In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import numpy as np

In [ ]:
# Preparing data
data = pd.read_csv('Fish.csv')
print('Fish dataset:\n')
print(data.sample(10))
print('\nPredict weight based on length1, length2, length3 and height:')
print('weight = w0 + w1*length1 + w2*length2 + w3*length + w4*height\n')
X = data.iloc[:, 2:].values
y = data.iloc[:, 1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
Xbar = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
Xbar_test = np.concatenate((np.ones((X_test.shape[0], 1)), X_test), axis=1)

Fish dataset:

       Species  Weight  Length1  Length2  Length3   Height   Width
138       Pike   567.0     43.2     46.0     48.7   7.7920  4.8700
81       Perch    85.0     18.2     20.0     21.0   5.0820  2.7720
158      Smelt    19.9     13.8     15.0     16.2   2.9322  1.8792
19       Bream   650.0     31.0     33.5     38.7  14.4738  5.7276
135       Pike   510.0     40.0     42.5     45.5   6.8250  4.4590
139       Pike   770.0     44.8     48.0     51.2   7.6800  5.3760
84       Perch   125.0     19.0     21.0     22.5   5.6925  3.6675
71      Parkki   300.0     24.0     26.0     29.0  11.3680  4.2340
60   Whitefish  1000.0     37.3     40.0     43.5  12.3540  6.5250
42       Roach   120.0     19.4     21.0     23.7   6.1146  3.2943

Predict weight based on length1, length2, length3 and height:
weight = w0 + w1*length1 + w2*length2 + w3*length + w4*height



In [ ]:
# Solution using gradient descent (Nesterov Accelerated Gradient)
m = Xbar.shape[0]
y_mat = np.reshape(y_train, (y_train.shape[0], 1))


def cost(w):
    return 0.5 / m * (np.linalg.norm(y_mat - Xbar.dot(w)) ** 2)


def grad(w):
    g = 1 / m * Xbar.T.dot(Xbar.dot(w) - y_mat)
    return g


def GD_NAG(grad, w_init, eta=0.02, gamma=0.9, max_iter=400):
    # print('Running gradient descent...')
    w = [w_init]
    v = [np.zeros_like(w_init)]

    for it in range(max_iter):
        v_new = gamma * v[-1] + eta * grad(w[-1] - gamma * v[-1])
        w_new = w[-1] - v_new
        w.append(w_new)
        v.append(v_new)
        if np.linalg.norm(grad(w_new)) / np.array(w_init).size < 1e-4:
            break
        # print('Epoch {}/{}, Loss: {}\nw = {}'.format(it, max_iter, cost(w_new), w[-1].T))
    return w[-1]


w_init = np.random.random((Xbar.shape[1], 1))
w = GD_NAG(grad, w_init, eta=0.0004, max_iter=int(1e6))
print("Solution using gradient descent:\nw = {}".format(w.reshape((w.shape[0],))))

pred = Xbar_test.dot(w)
print("Accuracy score: {} %\n".format(100 * r2_score(y_test, pred)))

Solution using gradient descent:
w = [-471.89530955   54.30102433    3.89862602  -33.1868454    31.69170561
   18.18389469]
Accuracy score: 90.28693417212172 %



In [ ]:
# Solution using normal equation
w = np.dot(np.linalg.pinv(np.dot(Xbar.T, Xbar)), np.dot(Xbar.T, y_train))
pred = np.dot(Xbar_test, w)
print("Solution using normal equation:\nw = {}".format(w))
print("Accuracy score: {} %\n".format(100 * r2_score(y_test, pred)))

Solution using normal equation:
w = [-471.8972231    54.28670501    3.9136931   -33.18816788   31.69150649
   18.18219128]
Accuracy score: 90.28688015008512 %



In [ ]:
# Solution using scikit-learn
regr = LinearRegression()
regr.fit(X_train, y_train)
print("Solution using scikit-learn:\nw = {}".format(np.concatenate((np.array([regr.intercept_]), regr.coef_))))
print("Accuracy score: {} %\n".format(100 * regr.score(X_test, y_test)))

Solution using scikit-learn:
w = [-471.8972231    54.28670501    3.9136931   -33.18816788   31.69150649
   18.18219128]
Accuracy score: 90.2868801500446 %

